In [ ]:
from my_functions import *

In [ ]:
net_model = 'EfficientNetAutoAttB4ST'
train_db = 'DFDC'

device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 128

facedet = BlazeFace().to(device)
facedet.load_weights("../blazeface/blazeface.pth")
facedet.load_anchors("../blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)

In [ ]:
# #save features
device = torch.device('cuda:0')

model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]
net = getattr(fornet,net_model)().eval().to(device)
net.load_state_dict(load_url(model_url,map_location=device,check_hash=True))
transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)
names = ['CH','BO', 'RF', 'KP', 'EC', 'TB', 'JP', 'LN', 'MS', 'ND'] 

for name in names:
    for idx in range(5):      
        recon_path = './reconstructed_videos/'
        real_videos_train, real_videos_valid, real_videos_test = [], [], []
        fake_videos_train, fake_videos_valid, fake_videos_test = [], [], []
        mypath = './celeb_videos/' + name + '/'
        video_files = []
        for (dirpath, dirnames, filenames) in walk(mypath):
            video_files.extend(filenames)
            break
        i = idx
        
        capture = cv2.VideoCapture(mypath + video_files[i])
        frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        video_read_fn = lambda x: videoreader.read_frames(x, frame_count, jitter=0, seed=None)
        face_extractor = FaceExtractor(video_read_fn=video_read_fn,facedet=facedet)
        vid_real_faces = face_extractor.process_video(mypath + video_files[i])
        faces_real_t = torch.stack( [ transf(image=frame['faces'][0])['image'] for frame in vid_real_faces if len(frame['faces'])] )

        vid_name = video_files[i][0:3] + 'FSG_all_recon1.mp4'
        vid_faces = face_extractor.process_video(recon_path + vid_name)
        faces_real_recon = torch.stack( [ transf(image=frame['faces'][0])['image'] for frame in vid_faces if len(frame['faces'])] )
        with torch.no_grad():
            out1 = net.features(faces_real_t.to(device))
            out2 = net.features(faces_real_recon.to(device))
        with open('./saved_features/features_real_'+name+'_v_'+str(idx)+'_recon_all_fpv'+str(frames_per_video)+'.pkl', 'wb') as f: pickle.dump([out1, out2], f) #save results
            
            
        mypath = './celeb_videos/fake_' + name + '/'
        video_files = []
        for (dirpath, dirnames, filenames) in walk(mypath):
            video_files.extend(filenames)
            break
        i = idx
        capture = cv2.VideoCapture(mypath + video_files[i])
        frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        video_read_fn = lambda x: videoreader.read_frames(x, frame_count, jitter=0, seed=None)
        face_extractor = FaceExtractor(video_read_fn=video_read_fn,facedet=facedet)
        vid_fake_faces = face_extractor.process_video(mypath + video_files[i])
        faces_fake_t = torch.stack( [ transf(image=frame['faces'][0])['image'] for frame in vid_fake_faces if len(frame['faces'])] )
        
        vid_name = video_files[i]
        vid_name = vid_name[:vid_name.index(".")]
        vid_name = vid_name + 'FSG_all_recon1.mp4'
        vid_fake_faces = face_extractor.process_video(recon_path + vid_name)
        faces_fake_recon = torch.stack( [ transf(image=frame['faces'][0])['image'] for frame in vid_fake_faces if len(frame['faces'])] )
        with torch.no_grad():
            out1 = net.features(faces_fake_t.to(device))
            out2 = net.features(faces_fake_recon.to(device))
        with open('./saved_features/features_fake_'+name+'_v_'+str(idx)+'_recon_all_fpv'+str(frames_per_video)+'.pkl', 'wb') as f: pickle.dump([out1, out2], f) #save results
